<div
    style='background-image: url("images/_banner_.jpg"); padding: 0px;
    background-size: cover; border-radius: 10px; height: 250px;
    background-position: 50% 50%'>
    <div
        style="float: left; margin: 20px; padding: 10px;
        background: rgba(255 , 255 , 255 , 0.8); width: 40%; height: 100px;
        border-radius: 10px">
        <div
            style="position: relative; top: 50%;
            transform: translatey(-50%)">
            <div
                style="font-size: xx-large; font-weight: 900;
                color: rgba(0 , 0 , 0 , 0.9);
                line-height: 100%">Python Workshop
            </div>
            <div
                style="font-size: large; padding-top: 20px;
                color: rgba(0 , 0 , 0 , 0.7)">Python crash course
            </div>
        </div>
    </div>
    <div style="float: right; font-size: small;
        position: relative; top: 90%; padding: 5px;
        background: rgba(255 , 255 , 255 , 0.6); border-top-left-radius: 10px;
        color: rgba(0 , 0 , 0 , 1)">Shahar Shani-Kadmiel, CiTG, TU Delft
    </div>
</div>

This notebook (inspired by [Lion Krischer](https://github.com/krischer)) is a very quick introduction to Python and in particular its scientific ecosystem in case you have never seen it before. It furthermore grants a possibility to get to know the [IPython/Jupyter notebook](http://www.nature.com/news/interactive-notebooks-sharing-the-code-1.16261). [See here for the official documentation](http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb) of the Jupyter notebook - a ton more information can be found online.


A lot of motivational writing on *Why Python?* is out there so we will not repeat it here and just condense it to a single sentence:

***Python is a good and easy to learn, open-source, general purpose programming language that happens to be very good for many scientific tasks (due to its vast scientific ecosystem).***

---

## Quick Reference on How to Use This Notebook


![toolbar](images/toolbar.png)

* <kbd>Shift</kbd> + <kbd>Enter</kbd> (<kbd>⇧↩</kbd>): Execute cell and jump to the next cell
* <kbd>I</kbd>,<kbd>I</kbd>: Interrupt execution
* <kbd>0</kbd>,<kbd>0</kbd>: Restart kernel
* <kbd>A</kbd>: Insert cell above
* <kbd>B</kbd>: Insert cell below
* <kbd>D</kbd>,<kbd>D</kbd>: Delete cell

Keyboard shortcuts are useful as they save you time pointing your cursor to click buttons. It is worth spending some time to learn them. To see all available shortcuts (and even edit them, *if you really want to*) hit <kbd>H</kbd>.

---
### Exercise

This is a code cell, execute it. Try to add a cell above it and below it.

In [ ]:
# execute this cell by pressing shift + enter
print('Welcome to this fabulous workshop!')

### Notebook vs. Script and CLI

In this workshop we will mostly work inside Jupyter-Notebooks but these are only one way of using Python. Writing scripts to text files and executing them with the Python interpreter of course also works:

```bash
$ python some_script.py
```

Another alternative is interactive usage on the command line:

```bash
$ ipython
```

We will see some tricks to combine all three in the prototyping phase of a project.

---
### *Notebooks are excellent for teaching!*

#### Curve fitting interactive example:

In [ ]:
%matplotlib notebook
from ipywidgets import interactive, fixed, HBox, VBox

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit


def f(x, a, mu, sigma):
    """
    PDF of a normal distribution.
    
    Parameters
    ----------
    x : array
        Values for which to calculate P(x).
        
    a : float
        Amplitude of the function.
        
    mu : float
        The mean of the distribution.
        
    sigma : float
        The standard deviation.
    
    Returns
    -------
    P(x) : array
        The normal distribution PDF.
    """
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2))


def make_data(x, a=1, mu=0, sigma=1, n=25, noise=0.1):
    """
    Generate some data by modeling the normal distribution PDF and
    adding noise.
    
    Parameters
    ----------
    x : array
        Values for which to calculate P(x).
        
    a : float, optional
        Amplitude of the function.
        
    mu : float, optional
        The mean of the distribution.
        
    sigma : float, optional
        The standard deviation.
        
    n : int, optional
        Number of data points to generate.
        
    niose : float, optional
        Fraction of the amplitude to add to teh ideal data.
    
    Returns
    -------
    P_noisy(x) : array
        The normal distribution PDF with noise added.
    """
    data = f(x, a, mu, sigma) + noise * a * (np.random.rand(n) * 2 - 1)
    return data
    

def fit(xy_data, x_model, model, p0=None):
    """
    Wrapper of SciPy's curve_fit function.
    
    Parameters
    ----------
    xy_data : tuple
        A tuple of 2x1d arrays with x, y data to fit.
        
    x_model : array
        An array of x points at which the fitted function will be
        evaluated.
        
    model : callable
        The model function to use for fitting.
        
    p0 : array_like, optional
        An initial guess for the fitting parameters.
        
    Returns
    -------
    P_fitted(x) : tuple
        P at points x_model evaluated by the fitted curve.
        
    p_opt : array
        An array of the optimized parameters in the curve fitting.
        
    p_cov : array
        Covariance matrix of the parameters.
    """
    p_opt, p_cov = curve_fit(model, *xy_data, p0=p0)
    return model(x_model, *p_opt), p_opt, p_cov

def init_plot(xy_data, xy_model, xy_ideal):
    """
    Initialize the figure axes and canvas with data. The scatter points
    and lines will then be updated from an interactive function that
    does not regenerate the entire figure in every interaction.
    
    Parameters
    ----------
    xy_data, xy_model, xy_ideal : tuples
        Tuples of 2x1d arrays of x, y points representing the data, the
        fitted model, and the ideal model, respectively.
        
    Returns
    -------
    fig, ax, sp_data, l_fit, l_model :
        figure and axes instances, scatter-point instance of the noisy
        data, line instance of the fitted curve, line instance of the
        ideal curve.
    """
    
    fig, ax = plt.subplots(figsize=(7, 5))
    sp_data = ax.scatter(*xy_data, s=20, c='none', edgecolors='k', label='Data')

    l_fit, = ax.plot(*xy_model, label='Fit')
    l_model, = ax.plot(*xy_ideal, '--', label='Model')
    
    ax.grid()
    ax.legend(loc=8, ncol=3, handletextpad=0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('P(x)')
    ax.set_title('Curve Fitting is Fun!')
    
    return fig, ax, sp_data, l_fit, l_model

a = 1
mu = 0
sigma = 1
n = 25
noise = 0.1

xmin, xmax = mu - 3 * sigma, mu + 3 * sigma
x_data = (np.random.rand(n) * 2 - 1) * 0.5 * (xmax - xmin) + mu
y_data = make_data(x_data, a, mu, sigma, n, noise)

x_model = np.linspace(xmin, xmax, 2 * n)

y_model, p_opt, p_cov = fit((x_data, y_data), x_model, f)

y_ideal = f(x_model, a, mu, sigma)

fig, ax, sp_data, l_fit, l_model = init_plot((x_data, y_data),
                                             (x_model, y_model),
                                             (x_model, y_ideal))

def update_plot(fig, ax, sp_data, l_fit, l_model,
                a=1, mu=0, sigma=1, n=25, noise=0.1):
    xmin, xmax = mu - 3 * sigma, mu + 3 * sigma
    x_data = (np.random.rand(n) * 2 - 1) * 0.5 * (xmax - xmin) + mu
    y_data = make_data(x_data, a, mu, sigma, n, noise)
    
    sp_data.set_offsets(np.column_stack((x_data, y_data)))
    pad = y_data.ptp() * 0.05
    ax.set_ylim(y_data.min() - pad, y_data.max() + pad)
    ax.set_xlim(xmin, xmax)

    x_model = np.linspace(xmin, xmax, 2 * n)
    y_model, p_opt, p_cov = fit((x_data, y_data), x_model, f)
    
    l_fit.set_data(x_model, y_model)
    l_model.set_data(x_model, f(x_model, a, mu, sigma))

    print('{:<15} {:>15} {:>15}'.format('', 'Model params', 'Fit params'))
    print('{:<15} {:15} {:15.5f}'.format('Amplitude', a, p_opt[0]))
    print('{:<15} {:15} {:15.5f}'.format('Mean', mu, p_opt[1]))
    print('{:<15} {:15} {:15.5f}'.format('Stand. Dev.', sigma, p_opt[2]))
    

w = interactive(
    update_plot, fig=fixed(fig), ax=fixed(ax),
    sp_data=fixed(sp_data), l_fit=fixed(l_fit), l_model=fixed(l_model),
    a=(0.5, 5, 0.5), mu=(-5, 5, 0.5), sigma=(0, 2, 0.2), n=(10, 100, 1),
    noise=(0, 1, 0.1))

controls = VBox([HBox(w.children[:-1]), w.children[-1]])
controls.children[0].layout.flex_flow = 'row wrap'
controls.children[0].layout.justify_content = 'center'
controls.layout.flex_flow = 'row wrap'
controls.layout.justify_content = 'center'
controls
# w

#### Superposition of Sines interactive example:

In [ ]:
%matplotlib notebook
from ipywidgets import interactive, fixed, HBox, VBox

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

def sine(x, a=1, f=1, phase=0):
    phase = np.radians(phase)
    return a * np.sin(2 * np.pi * f * (x - phase))


def spectrum(signal, dt):
    freq = np.fft.rfftfreq(signal.size, dt)
    amp = np.abs(np.fft.rfft(signal)) * 2 / signal.size
    return freq, amp


def make_data(x_bound=2 * np.pi, dt=0.05,
              a1=1, f1=1, phase1=0,
              a2=0.5, f2=0.5, phase2=0):
    
    x = np.arange(-x_bound, x_bound, dt)
    
    signal1 = sine(x, a1, f1, phase1)
    signal2 = sine(x, a2, f2, phase2)
    signal3 = signal1 + signal2
    
    freq, amp1 = spectrum(signal1, dt)
    freq, amp2 = spectrum(signal2, dt)
    freq, amp3 = spectrum(signal3, dt)
    
    return x, signal1, signal2, signal3, freq, amp1, amp2, amp3
    

def init_plot(x, signal1, signal2, signal3,
              freq, amp1, amp2, amp3):
    
    fig, ax = plt.subplots(
        2, 2, sharex='col', sharey='col',
        gridspec_kw=dict(bottom=0.15, hspace=0.06, wspace=0.03,
                         width_ratios=(2, 1)),
        figsize=(7, 5))
    
    ((ax1, ax11), (ax2, ax22)) = ax

    l_s3, = ax1.plot(x, signal3, 'k')
    ax1.yaxis.tick_left()
    ax1.set_title('Signals')

    
    l_fs3, = ax11.semilogx(freq, amp3, 'k')
    ax11.grid(True, which='both', lw=0.5)
    ax11.yaxis.tick_right()
    ax11.set_title('Fourier Spectrum')

    l_s1, l_s2 = ax2.plot(x, signal1, x, signal2)
    ax2.yaxis.tick_left()
    ax2.set_xlabel('Time, s')
    ax2.set_ylabel('Amplitude')

    
    l_fs1, l_fs2 = ax22.semilogx(freq, amp1, freq, amp2)
    ax22.grid(True, which='both', lw=0.5)
    ax22.set_xlabel('Frequency, Hz')
    ax22.yaxis.tick_right()
    ax22.yaxis.set_label_position('right')
    ax22.set_ylabel('Amplitude')
    
    return fig, ax, l_s1, l_s2, l_s3, l_fs1, l_fs2, l_fs3


label_template = 'a={}, f={}, phase={}'
fig, ax, l_s1, l_s2, l_s3, l_fs1, l_fs2, l_fs3 = init_plot(
    *make_data())


def update_plot(fig, ax, l_s1, l_s2, l_s3, l_fs1, l_fs2, l_fs3,
                x_bound=2 * np.pi, dt=0.05,
                a1=1, f1=1, phase1=0, a2=0.5, f2=0.5, phase2=0):
    
    x, signal1, signal2, signal3, freq, amp1, amp2, amp3 = make_data(
        x_bound, dt, a1, f1, phase1, a2, f2, phase2)

    
    l_s1.set_data(x, signal1)
    l_s2.set_data(x, signal2)
    l_s3.set_data(x, signal3)
    l_fs1.set_data(freq, amp1)
    l_fs2.set_data(freq, amp2)
    l_fs3.set_data(freq, amp3)
    
    l_s1.set_label(label_template.format(a1, f1, phase1))
    l_s2.set_label(label_template.format(a2, f2, phase2))
    
    ((ax1, ax11), (ax2, ax22)) = ax
    ax1.set_xlim(x[0], x[-1])
    ax11.set_xlim(freq[1], freq[-1])
    
    ax1.set_ylim(signal3.min(), signal3.max())
    ax11.set_ylim(amp3.min(), amp3.max())
    
    ax2.legend([label_template.format(a1, f1, phase1),
                label_template.format(a2, f2, phase2)],
               loc=9, bbox_to_anchor=(0.5, -0.25), ncol=2)
    
    

w = interactive(
    update_plot, fig=fixed(fig), ax=fixed(ax),
    l_s1 = fixed(l_s1), l_s2 = fixed(l_s2), l_s3 = fixed(l_s3),
    l_fs1 = fixed(l_fs1), l_fs2 = fixed(l_fs2), l_fs3 = fixed(l_fs3),
    x_bound=(2 * np.pi, 20 * np.pi, np.pi), dt=(0.001, 0.1, 0.001),
    a1=(0, 10, 0.5), f1=(0.1, 2, 0.1), phase1=(0, 360, 15),
    a2=(0, 10, 0.5), f2=(0.1, 2, 0.1), phase2=(0, 360, 15))

controls = VBox([HBox(w.children[:2]),
                 HBox([VBox(w.children[2:5]), VBox(w.children[5:])])])
controls.layout.align_items = 'center'
controls

---

## Useful Links

Here is collection of resources regarding the scientific Python ecosystem. They cover a number of different packages and topics; way more than we will manage today.

If you have any question regarding some specific Python functionality you can consult the official [Python documentation](http://docs.python.org/).
 
Furthermore a large number of Python tutorials, introductions, and books are available online. Here are some examples for those interested in learning more.
 
* [Learn Python The Hard Way](http://learnpythonthehardway.org/book/)
* [Dive Into Python](http://www.diveintopython.net/)
* [The Official Python Tutorial](http://docs.python.org/2/tutorial/index.html)
* [Think Python Book](http://www.greenteapress.com/thinkpython/thinkpython.html)
 
Some people might be used to Matlab - this helps:
 
* [NumPy for Matlab Users Introdution](http://wiki.scipy.org/NumPy_for_Matlab_Users)
* [NumPy for Matlab Users Cheatsheet](http://mathesaurus.sourceforge.net/matlab-numpy.html)
 
 
Additionally there is an abundance of resources introducing and teaching parts of the scientific Python ecosystem.
 
* [NumPy Tutorial](http://wiki.scipy.org/Tentative_NumPy_Tutorial)
* [Probabilistic Programming and Bayesian Methods for Hackers](http://camdavidsonpilon.github.io/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/): Great ebook introducing Bayesian methods from an understanding-first point of view with the examples done in Python.
* [Python Scientific Lecture Notes](http://scipy-lectures.github.io/): Introduces the basics of scientific Python with lots of examples.
* [Python for Signal Processing](http://python-for-signal-processing.blogspot.de/): Free blog which is the basis of a proper book written on the subject.
* [Another NumPy Tutorial](http://www.loria.fr/~rougier/teaching/numpy/numpy.html), [Matplotlib Tutorial](http://www.loria.fr/~rougier/teaching/matplotlib/matplotlib.html)
 
You might eventually have a need to create some custom plots. The quickest way to success is usually to start from some example that is somewhat similar to what you want to achieve and just modify it. These websites are good starting points:
 
* [Matplotlib Gallery](http://matplotlib.org/gallery.html)
* [ObsPy Gallery](http://docs.obspy.org/gallery.html)
* [Basemap Gallery](http://matplotlib.org/basemap/users/examples.html)

---

## Core Python Crash Course

This part is fairly non-interactive and serves to get you up to speed with Python assuming you have practical programming experience with at least one other language. Nonetheless please change things and play around on your own - it is the only way to really learn it!

### Numbers

Python is dynamically typed and assigning something to a variable will give it that type.

In [ ]:
# Three basic types of numbers
a = 1             # Integers
b = 2.0           # Floating Point Numbers
c = 3.0 + 4j      # Complex Numbers, note the use of j for the complex part

print(type(a), type(b), type(c))

# Arithmetics work as expected.
# Upcasting from int -> float -> complex
d = a + b         # (int + float = float)
print(d)

e = c**2        # c to the second power, performs a complex multiplication
print(e)

### Strings

Just enclose something in single or double quotes and it will become a string. On Python 3 it defaults to unicode strings, e.g. non Latin alphabets and other symbols.

In [ ]:
# You can use single or double quotes to create strings.
location = "Delft, The Netherlands"

# Concatenate strings with plus.
where_am_i = 'I am in ' + location + '.'

# Print things with the print() function.
print(location, 1, 2, 3 / 2)
print(where_am_i)

# Strings have a lot of attached methods for common manipulations.
print(location.lower())
print(location.upper())

# Square bracket are used for indexing. Negative indices are from the back.
print(location[0], location[-1])

# ...and slicing.
print(where_am_i[5: 22])
print(where_am_i[5: -5])

#### Exercise

Assign your full name in **all** lower-case letters to a variable, and print it, capitalizing the first letter of each part. *Protip: [Google for "titlecase a string in python"](https://www.google.com/search?&q=titlecase+a+string+in+python)*. This works for almost any programming problem - someone will have had the same issue before!

In [ ]:
# enter code here (2 lines)



### Collections

Python has three main collection types: Tuples, Lists and Dictionaries.

#### Tuples

In [ ]:
# Tuples use round brackets and are simple ordered collections of things.
tuple_everything = ('hello', a, b, c, 1, 2, 3)
print(type(tuple_everything))

# Access elements with the same slicing/indexing notation as strings.
# Note that Python indices are zero based!
print(tuple_everything[0])
print(tuple_everything[:3])
print(tuple_everything[2:-2])

# Negative indices are counted from the back of the list.
print(tuple_everything[-3:])

Tuples are *immutable* (read-only). This means they cannot be changed:

In [ ]:
tuple_everything[0] = 'Hi'

#### Lists

In [ ]:
# List use square brackets and are simple ordered collections of things.
list_everything = ['hello', a, b, c, 1, 2, 3]
print(type(list_everything))

Tuples can turned into lists and visa versa:

In [ ]:
# they are not the same
print(list_everything == tuple_everything)

# but can be made the same
print(list_everything == list(tuple_everything))
print(tuple(list_everything) == tuple_everything)

Lists are *mutable*:

In [ ]:
print(list_everything)

# assigning at indices
list_everything[0], list_everything[-1] = 'Olla', 3**3
print(list_everything)

# Appending with the append method
list_everything.append("you")
print(list_everything)

# or adding two lists together
list_everything += [10, 20, 30, 'some text']
print(list_everything)

#### Exercise

Swap the first and last items in the following list and print the result.

In [ ]:
# the list
stuff = ['this', 'comes', 'before', 'that']

# enter code here (approx 2 lines)



#### Dictionaries

Each entry is a key-value pair. Dictionaries are unordered.

In [ ]:
# Dictionaries have named fields and no inherent order. As is
# the case with lists, they can contain anything.
information = {
    'name': 'John',
    'surname': 'Doe',
    'age': 30,
    'kids': ['Jonny', 'Janie']
}

print(type(information))

# Acccess items by using the key in square brackets.
print(information['kids'])

# Add new things by just assigning to a key.
print(information)

information['favorite music'] = 'jazz'
print(information)

# Delete things by using the del operator
del information['age']
print(information)

### Imports

To use non built-in functions and objects not part of the default namespace, you have to import them. You will have to do this a lot so it is necessary to learn how to do it.

In [ ]:
# Import anything, and use it with the dot accessor.
import math

a = math.cos(4 * math.pi)

# You can also selectively import things.
from math import pi

b = 3 * pi

# And even rename them if you don't like their name.
from math import cos as cosine
c = cosine(b)

How to know what is available?

1. Read the [documentation](https://docs.python.org/3/library/math.html)
2. Interactively query the module

In [ ]:
print(dir(math))

Typing the **.** and the <kbd>TAB</kbd> will kick off tab-completion and allow you to interrogate the object.

In [ ]:
math.

In the IPython framework you can also use a question mark to view the documentation of modules and functions.

In [ ]:
math.cos?

### Control Flow

Loops and conditionals are needed for any non-trivial task. Please note that **whitespace matters in Python**. Everything that is indented at the same level is part of the same block. By far the most common loops in Python are for-each loops as shown in the following. While loops also exist but are rarely used.

In [ ]:
for i in range(len(list_everything)):
    print(list_everything[i])

In [ ]:
for item in list_everything:
    print(item)

In [ ]:
temp = ['a', 'b', 'c']

# The typical Python loop is a for-each loop, e.g.
for item in temp:
    # Everything with the same indentation is part of the loop.
    new_item = item + ' ' + item
    print(new_item)
    
print('No more part of the loop.')    

The second crucial control flow structure are if/else conditional and they work the same as in any other language.

In [ ]:
# If/else works as expected.
age = 77

if age >= 0 and age < 10:
    print("Younger than ten.")
elif age >= 10:
    print("Older than ten.")
else:
    print("wait what?")

*List comprehensions* are a nice way to write compact loops.

*Make sure you understand this as it is very common in Python.*

In [ ]:
a = range(10)
print(list(a))
b = [i for i in a if not i % 2]
print(b)

# Equivalant loop for b.
b = []
for i in a:
    if not i % 2:
        b.append(i)
print(b)

List comprehensions are **faster** too. Look:

In [ ]:
a = range(10000)
%timeit b = [i for i in a if not i % 2]

In [ ]:
%%timeit
b = []
for i in a:
    if not i % 2:
        b.append(i)

### Functions

The key to conquer a big problem is to divide it into many smaller ones and tackle them one by one. This is usually achieved by using functions.

In [ ]:
# Functions are defined using the def keyword.
def do_stuff(a, b):
    return a * b

# And called with the arguments in round brackets.
print(do_stuff(2, 3))

# Python function also can have optional arguments.
def do_more_stuff(a, b, power=1):
    return (a * b) ** power

print(do_more_stuff(2, 3))
print(do_more_stuff(2, 3, power=3))

# For more complex function it is oftentimes a good idea to 
# explicitly name the arguments. This is easier to read and less error-prone.
print(do_more_stuff(a=2, b=3, power=3))

#### Document your functions!

I cannot stress this enough. The whole idea of writing functions is to reuse them over and over again. If you need to read the code inside a function to know what it does, whoever wrote it did a bad job. Some functions are straightforward and need only one line of documentation, others need more comprehensive documentation.

In [ ]:
# this is a straightforward function
def fibonacci_expander(n):
    """
    Return a list of the first ``n`` elements of the Fibonacci series.
    """
    result = [0, 1]
    if n < 3:
        raise ValueError(
            'The first two elements of the Fibonacci series are 0, 1. Try n > 2.')
        
    for n_ in range(2, n):
        result.append(result[-1] + result[-2])
    return result


def fibonacci_expander2(n=None, vmax=None):
    """
    A Fibonacci series expander.
    
    Parameters
    ----------
    n : {int, None}, optional
        Return the first ``n`` elements of the Fibonacci series.
        
    vmax : {int, None}, optional
        Return all elements of the Fibonacci series smaller than
        or equal to ``vmax``.
        
    Note
    ----
    ``n`` is ignored if ``vmax`` is specified.
        
    Return
    ------
    result : list
        First ``n`` elements or elements smaller than or equal to
        ``vmax`` of the Fibonacci series.
        
    Examples
    --------
    >>> fibonacci_expander2(10)
    [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
    
    >>> fibonacci_expander2(vmax=10)
    [0, 1, 1, 2, 3, 5, 8]
    """
    result = [0, 1]
    if vmax:
        while result[-1] <= vmax:
            result.append(result[-1] + result[-2])
        return result[:-1]
        
    else:
        return fibonacci_expander(n)

### Error Messages

You will eventually run into some error messages. Learn to read them! The last line is often the one that matters - reading upwards traces the error back in time and shows what calls led to it. If stuck: just google the error message!

In [ ]:
def do_something(a, b): 
    print(a + b + something_else)
    
do_something(1, 2)    

## Classes

We have already seen classes, *remember*? `int` is a class, `tuple` is a class. Classes are objects (*structures*) that contain values (*attributes*) and attached functions (*methods*). They are the cornerstone of OOP. We use classes in our everyday life without even thinking about it:

> Here is a piece of grey furniture. It is called a table. It has 4 legs. it is made out of wood and metal. it can be used for working on, hiding under, and drumming a beat.

What would this object look like as a class?

In [ ]:
class Furniture:
    """
    A top class binding all furnitures.
    """
    def __init__(self, color, made_of):
        self.color = color
        self.made_of = made_of
        
    def info(self):
        print('This is a piece of {} furniture made of {}.'.format(
            self.color, self.made_of))
        
    def drum_the_beat(self):
        print('1-and-2-and-3-and-4-and-1-and-2-and-3-and-4-and...')

        
class Table(Furniture):
    """
    This is a table.
    """
    def __init__(self, color, made_of, number_of_legs):
        Furniture.__init__(self, color, made_of)
        self.number_of_legs = number_of_legs
        
    def hide_under(self):
        print('Good, you are safe now.')
        
    def work(self):
        print("Take a coffee break. You've been working too hard.")
        
    def info(self):
        print('This is a {} legged {} table made of {}.'.format(
                self.number_of_legs, self.color, self.made_of))

In [ ]:
desk = Table('gray', 'wood and metal', 4)
desk.info()


Note that classes are very flexible objects. We can assign new attributes to an existing class:

In [ ]:
desk.price = dict(sum=100, currency='Euros')

desk.price

## The Scientific Python Ecosystem

The [SciPy Stack](https://www.scipy.org/stackspec.html) forms the basis for essentially all applications of scientific Python. Here we will quickly introduce the three core libraries:

* `NumPy`
* `SciPy`
* `Matplotlib`

The SciPy stack furthermore contains `pandas` (library for data analysis on tabular and time series data) and `sympy` (package for symbolic math), both very powerful packages, but we will omit them in this tutorial.

### NumPy

Large parts of the scientific Python ecosystem use NumPy, an array computation package offering N-dimensional, typed arrays and useful functions for linear algebra, Fourier transforms, random numbers, and other basic scientific tasks.

In [ ]:
import numpy as np

# Create a large array with with 1 million samples.
x = np.linspace(start=0, stop=100, num=1E6, dtype=np.float64)

# Most operations work per-element.
y = x ** 2

# Uses C and Fortran under the hood for speed.
print(y.sum())

# FFT and inverse
x = np.random.random(100)
large_X = np.fft.fft(x)
x = np.fft.ifft(large_X)

#### Something to note

When slicing and indexing, NumPy uses memory-views (pointers). This is very efficient memory-wise but can play tricks on you.

In [ ]:
# init array ``a`` with some values
a = np.arange(-100, 100, 10)
print(a)

# assign a subset of ``a`` to ``b``
b = a[7: 14]
print(b)

# change a value in ``b``
b[3] = 999
print(b)

That is expected, right? What are the array values stored in `a`?

In [ ]:
print(a)

So the values of `a` were not actually assigned to `b`, only a pointer to their location in memory. This means that when we change an element in `b`, we are actually touching memory-space that belongs to `a`.

This of course means, that changing an element in `a` will definitely cause a change in `b`.

In [ ]:
a[9] = 12345
print(a)
print(b)

In some cases this behavior is undesirable. Use the `copy()` method to explicitly copy the values themselves rather than *pointing* to them.

In [ ]:
a = np.arange(-100, 100, 10)
print(a)

# assign a subset of ``a`` to ``b``
b = a[7: 14].copy()
print(b)

# change a value in ``b``
b[3] = 999
print(b)

### Matplotlib

Plotting is done using `Matplotlib`, a package for greating high-quality static plots. It has an interface that mimics Matlab which many people are familiar with.

In [ ]:
import matplotlib.pyplot as plt

cycles = 4
x, dx = np.linspace(0, cycles * 2 * np.pi, 1024, retstep=True)
y = np.sin(x)

fig, (ax1, ax2) = plt.subplots(ncols=2)
ax1.plot(x, y, color="green", label="Some Curve")
ax1.legend(loc=1)

freq = np.fft.rfftfreq(x.size, dx)
amp = np.abs(np.fft.rfft(y)) / (0.5 * x.size)
ax2.semilogx(freq, amp, 'g')


In [ ]:
from mpl_toolkits.mplot3d import axes3d

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'), figsize=(7, 7))
X, Y, Z = axes3d.get_test_data(0.05)
ax.plot_surface(X, Y, Z, cmap='coolwarm', rstride=8, cstride=8, alpha=0.5)
ax.plot_wireframe(X, Y, Z, color='k', rstride=8, cstride=8, alpha=0.1)
cset = ax.contourf(X, Y, Z, zdir='z', offset=-100, cmap='coolwarm')
cset = ax.contourf(X, Y, Z, zdir='x', offset=-40, cmap='coolwarm')
cset = ax.contourf(X, Y, Z, zdir='y', offset=40, cmap='coolwarm')

ax.set_xlabel('X')
ax.set_xlim(-40, 40)
ax.set_ylabel('Y')
ax.set_ylim(-40, 40)
ax.set_zlabel('Z')
ax.set_zlim(-100, 100)
ax.set_aspect(1)

### SciPy

`SciPy`, in contrast to `NumPy` which only offers basic numerical routines, contains a lot of additional functionality needed for scientific work. Examples are solvers for basic differential equations, numeric integration and optimization, spare matrices, interpolation routines, signal processing methods, and a lot of other things.

In [ ]:
from scipy.interpolate import interp1d

# Under-sampled
x = np.linspace(0, 10, num=11)
y = np.cos(x ** 2 / 9.0)

# Over-sampled
x1 = np.linspace(0, 10, num=101)
y1 = np.cos(x1 ** 2 / 9.0)

# Cubic spline interpolation to new points.
y2 = interp1d(x, y, kind='cubic')(x1)

%matplotlib inline
plt.plot(x, y, 'o:', label='under-sampled')
plt.plot(x1, y1, '--', label='over-sampled')
plt.plot(x1, y2, label='interpolated')
plt.legend()

## Exercises

### Functions, NumPy, and Matplotlib

A. Write a function that takes a NumPy array `x` and `a`, `b`, and `c` and returns

$$
f(x) = a x^2 + b x + c
$$

B. Plot the result of that function with matplotlib.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# your code here: (approx 3 lines)




#### Bottles of Beer

*(stolen from http://www.ling.gu.se/~lager/python_exercises.html)*


"99 Bottles of Beer" is a traditional song in the United States and Canada. It is popular to sing on long trips, as it has a very repetitive format which is easy to memorize, and can take a long time to sing. The song's simple lyrics are as follows:

```
99 bottles of beer on the wall, 99 bottles of beer.
Take one down, pass it around, 98 bottles of beer on the wall.
```

The same verse is repeated, each time with one fewer bottle. The song is completed when the singer or singers reach zero.

Your task here is write a Python program capable of generating all the verses of the song.


In [ ]:
# your code here: (approx 3 lines)




#### Ceasar Cipher

*(stolen from http://www.ling.gu.se/~lager/python_exercises.html)*

In cryptography, a Caesar cipher is a very simple encryption techniques in which each letter in the plain text is replaced by a letter some fixed number of positions down the alphabet. For example, with a shift of 3, A would be replaced by D, B would become E, and so on. The method is named after Julius Caesar, who used it to communicate with his generals. ROT-13 ("rotate by 13 places") is a widely used example of a Caesar cipher where the shift is 13. In Python, the key for ROT-13 may be represented by means of the following dictionary:

```python
key = {'a':'n', 'b':'o', 'c':'p', 'd':'q', 'e':'r', 'f':'s', 'g':'t', 'h':'u', 
       'i':'v', 'j':'w', 'k':'x', 'l':'y', 'm':'z', 'n':'a', 'o':'b', 'p':'c', 
       'q':'d', 'r':'e', 's':'f', 't':'g', 'u':'h', 'v':'i', 'w':'j', 'x':'k',
       'y':'l', 'z':'m', 'A':'N', 'B':'O', 'C':'P', 'D':'Q', 'E':'R', 'F':'S', 
       'G':'T', 'H':'U', 'I':'V', 'J':'W', 'K':'X', 'L':'Y', 'M':'Z', 'N':'A', 
       'O':'B', 'P':'C', 'Q':'D', 'R':'E', 'S':'F', 'T':'G', 'U':'H', 'V':'I', 
       'W':'J', 'X':'K', 'Y':'L', 'Z':'M'}
```

Your task in this exercise is to implement an decoder of ROT-13. Once you're done, you will be able to read the following secret message:

```
Pnrfne pvcure? V zhpu cersre Pnrfne fnynq!
```

**BONUS:** Write an encoder!

In [ ]:
sentence = "Prnfne pvcure? V zhpu cersre Prnfne fnynq!"

key = {'a':'n', 'b':'o', 'c':'p', 'd':'q', 'e':'r', 'f':'s', 'g':'t', 'h':'u', 
       'i':'v', 'j':'w', 'k':'x', 'l':'y', 'm':'z', 'n':'a', 'o':'b', 'p':'c', 
       'q':'d', 'r':'e', 's':'f', 't':'g', 'u':'h', 'v':'i', 'w':'j', 'x':'k',
       'y':'l', 'z':'m', 'A':'N', 'B':'O', 'C':'P', 'D':'Q', 'E':'R', 'F':'S', 
       'G':'T', 'H':'U', 'I':'V', 'J':'W', 'K':'X', 'L':'Y', 'M':'Z', 'N':'A', 
       'O':'B', 'P':'C', 'Q':'D', 'R':'E', 'S':'F', 'T':'G', 'U':'H', 'V':'I', 
       'W':'J', 'X':'K', 'Y':'L', 'Z':'M'}

result = ""
# your code here: (approx 6 lines)







#end your code
print(result)